## Generative AI PDF information exctractor
This is my Master thesis project which results in an AI app to make a LLM from Open AI capable of answer specific questions about a topic. The model is able to access external data that hasn't seen during training, understand a user's question, exctract the information it needs to answer the user and return a comprehensive answer to the question formulated by the user. In the image below, there's a schema of the pipeline from user's query to final output:

In [6]:
# pip install python-dotenv
# pip insatall langchain
# pip instal pypdf
# pip install tiktoken
# pip install faiss-cpu

In [10]:
from dotenv import load_dotenv
import os

load_dotenv("apis.env")
# openai_api_key=os.getenv('OPENAI_API_KEY')

True

In [24]:
# pdf_path = "C:/Users/Nacho/Documents/MASTER/TFM/databases/pdfs/BOE-034_Codigo_Civil_y_legislacion_complementaria.pdf"
pdf_path = "C:/Users/Nacho/Documents/MASTER/TFM/databases/pdfs/COMMANDER RULES.pdf"

# Load pdf with external info not seen during training of the LLM
# from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader(pdf_path)
# pages = loader.load_and_split()

# # Connect your Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# root_dir = "/content/gdrive/My Drive/"

from PyPDF2 import PdfReader
reader = PdfReader(pdf_path)


In [25]:
# Read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [26]:
raw_text[:100]

'COMMANDER RULES  \nPhilosophy  \nBefore reading the rules below, please read  the philosophy  of comma'

In [27]:
from langchain.text_splitter import CharacterTextSplitter

# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [28]:
len(texts)

3

In [29]:
texts[0]

'COMMANDER RULES  \nPhilosophy  \nBefore reading the rules below, please read  the philosophy  of commander ; simply \nfollowing the rules is not sufficient to ensure a good play experience.  \nDeck Construction Rules  \n1. Players choose a legendary creature as the  commander  for their deck.   \n2. A card’s  color  identity  is its color plus the color of any mana symbols in \nthe card’s rules text. A card’s color identity is established before the \ngame begins, and cannot be changed by game effects. The cards in a deck \nmay not have any colors in their color identity which are no t in the color \nidentity of the deck’s commander.  \n3. A Commander deck must contain exactly 100 cards, including the \ncommander. If you’re playing a companion, it must adhere to color \nidentity and singleton rules. While it is not part of the deck, it is \neffectively a 101st card.  \n4. With the exception of basic lands, no two cards in the deck may have the'

In [8]:
# Generate vector space representation with words from the external data
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [36]:
# Load embeddings in vector database
from langchain.vectorstores import FAISS
# db = FAISS.from_documents(pages, embeddings)
db = FAISS.from_texts(texts, embeddings)

# from langchain.vectorstores import Chroma
# db = Chroma.from_documents(pages, embeddings)
# db = Chroma.from_texts(texts, embeddings)

# q = "What is Link's traditional outfit color?"
# db.similarity_search(q)[0]

In [37]:
# Use information retrieval from embedding for answer
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI()
# chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
# q = "Qué sabes"
# chain(q, return_only_outputs=True)

chain = load_qa_chain(llm, chain_type="stuff")

In [41]:
query = "What are the restrictions for a card to be the commander?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The commander must be a legendary creature, and its color identity must match that of the deck, with no more than one card of the same name in the deck.'